# Lesson 5: Multiple Tables and Relationships

**Duration:** 20 minutes  
**Prerequisites:** Complete Lessons 1-4  
**Learning Mode:** Read explanations, then run each SQL query

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
- Understand database normalization and why it matters
- Design one-to-many relationships
- Create related tables with foreign keys
- Link tables together properly
- Avoid data duplication
- Build many-to-many relationships with junction tables


## 📚 Why Use Multiple Tables?

Currently, our `characters` table stores homeworld as TEXT. This creates problems:

### Problems with Single Table Design:

1. **Duplication:** "Tatooine" is stored multiple times
2. **Inconsistency:** One entry might say "Tatooine", another "tatooine"
3. **Limited information:** We can't store climate, population, etc.
4. **Update issues:** Changing planet name requires updating many rows

### Solution: Use Multiple Related Tables!

Instead of storing planet names repeatedly, we:
- Create a `planets` table with all planet information
- Store only a planet ID in the `characters` table
- Link characters to planets using that ID


## 🏗️ Understanding Database Relationships

### One-to-Many Relationship

**Definition:** One record in Table A relates to many records in Table B.

**Examples:**
- **One planet → Many characters** (many characters from one planet)
- One customer → Many orders
- One author → Many books

### Many-to-Many Relationship

**Definition:** Multiple records in Table A relate to multiple records in Table B.

**Examples:**
- **Many characters → Many vehicles** (characters pilot multiple vehicles, vehicles have multiple pilots)
- Many students → Many courses
- Many actors → Many movies


## 🔑 Primary Key vs Foreign Key

| Key Type | Purpose | Example |
|----------|---------|---------|
| **Primary Key** | Uniquely identifies each row in a table | `id` in `planets` table |
| **Foreign Key** | References a primary key in another table | `homeworld_id` in `characters` table |

### Visual Example:

```
characters table:
id | name           | homeworld_id (FOREIGN KEY)
1  | Luke Skywalker | 1
2  | Darth Vader    | 1

planets table:
id (PRIMARY KEY) | name     | climate
1                | Tatooine | arid
2                | Alderaan | temperate
```

Both Luke and Vader have `homeworld_id = 1`, which points to Tatooine.


## 🛠️ Setup: Connect to Database

**Run the next 2 cells:**

In [2]:
# Load SQL magic extension
%load_ext sql

# Fix prettytable compatibility issue
import prettytable
try:
    # Try to access DEFAULT to see if it exists
    _ = prettytable.DEFAULT
except AttributeError:
    # If it doesn't exist, add it using SINGLE_BORDER
    from prettytable import SINGLE_BORDER
    prettytable.DEFAULT = SINGLE_BORDER

# Configure SQL magic settings
%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

/tmp/ipykernel_635/3162060372.py:8: DeprecationWarning: the 'DEFAULT' constant is deprecated, use the 'TableStyle' enum instead
  _ = prettytable.DEFAULT


In [3]:
%sql sqlite:///starwars.db

## 📚 Schema Design Reference

**Tables you'll create:**
1. **planets** - Tatooine, Alderaan, Kashyyyk, Naboo, etc.
   - Columns: id, name, climate, population
   
2. **vehicles** - X-wing, Millennium Falcon, Speeder bike, etc.
   - Columns: id, name, model, manufacturer, cost

3. **characters** - Links to planets via foreign key
   - Columns: id, name, species, homeworld (references planets.id), height

4. **character_vehicles** - Junction table for many-to-many
   - Columns: character_id, vehicle_id

💡 **This lesson:** You'll build relationships between these tables!


### 📋 Sample Data Examples

**Planet examples to insert:**
- Tatooine (desert climate, ~200,000 population)
- Alderaan (temperate, ~2,000,000,000)
- Kashyyyk (tropical, ~45,000,000)
- Naboo (temperate, ~4,500,000,000)

**Vehicle examples to insert:**
- X-wing (Starfighter, Incom Corporation, 149,999 credits)
- Millennium Falcon (YT-1300 light freighter, Corellian, 100,000 credits)
- Speeder bike (74-Z speeder bike, Aratech, 8,000 credits)

**Character examples with foreign keys:**
- Luke Skywalker (Human, homeworld_id=1 for Tatooine, 172cm)
- Leia Organa (Human, homeworld_id=2 for Alderaan, 150cm)
- Chewbacca (Wookiee, homeworld_id=3 for Kashyyyk, 228cm)

**Character-Vehicle relationships:**
- Luke flies X-wing (character_id=1, vehicle_id=1)
- Han Solo flies Millennium Falcon

💡 **You'll create these tables and insert this data yourself!**


## 🪐 Part 1: Create the Planets Table

### Create Planets Table with Schema

**New Concept:** `UNIQUE` constraint ensures no duplicate planet names.


In [8]:
%%sql
-- Create the planets table
CREATE TABLE IF NOT EXISTS planets (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    climate TEXT,
    terrain TEXT,
    population INTEGER
);

[]

### Insert Planet Data


In [9]:
%%sql
-- Insert planets
INSERT INTO planets (name, climate, terrain, population) VALUES
('Tatooine', 'arid', 'desert', 200000),
    ('Alderaan', 'temperate', 'grasslands, mountains', 2000000000),
    ('Hoth', 'frozen', 'tundra, ice caves', NULL),
    ('Kashyyyk', 'tropical', 'jungle, forests', 45000000),
    ('Naboo', 'temperate', 'grassy hills, swamps', 4500000000),
    ('Corellia', 'temperate', 'plains, urban', 3000000000),
    ('Stewjon', 'temperate', 'grass', NULL),
    ('Unknown', NULL, NULL, NULL);


[]

### View Planets Table


In [11]:
%%sql
-- View the schema for planets table
SELECT * FROM planets;

id,name,climate,terrain,population
1,Tatooine,arid,desert,200000
2,Alderaan,temperate,"grasslands, mountains",2000000000
3,Hoth,frozen,"tundra, ice caves",None
4,Kashyyyk,tropical,"jungle, forests",45000000
5,Naboo,temperate,"grassy hills, swamps",4500000000
6,Corellia,temperate,"plains, urban",3000000000
7,Stewjon,temperate,grass,None
8,Unknown,None,None,None


## 🔗 Part 2: Link Characters to Planets

### Add homeworld_id Column to Characters

**Note:** This creates the foreign key reference.


In [12]:
%%sql
-- so apparently these comments are wrong about what we have to do. again. 
ALTER TABLE characters ADD COLUMN homeworld_id INTEGER;

[]

### Update Characters with Planet IDs

**Explanation:** These subqueries find each planet's ID and update the character's `homeworld_id`.

**Run the next cell to link all characters to their planets:**


In [13]:
%%sql
-- Create the characters table with foreign key
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Tatooine') WHERE homeworld = 'Tatooine';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Alderaan') WHERE homeworld = 'Alderaan';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Corellia') WHERE homeworld = 'Corellia';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Kashyyyk') WHERE homeworld = 'Kashyyyk';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Stewjon') WHERE homeworld = 'Stewjon';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Naboo') WHERE homeworld = 'Naboo';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Unknown') WHERE homeworld = 'Unknown';


[]

### Verify the Links


In [14]:
%%sql
-- View the characters schema
SELECT * FROM characters

id,name,species,homeworld,height,affiliation,homeworld_id
1,Luke Skywalker,Human,Tatooine,172,Rebel Alliance,1
2,Leia Organa,Human,Alderaan,150,Rebel Alliance,2
3,Han Solo,Human,Corellia,180,Rebel Alliance,6
4,Chewbacca,Wookiee,Kashyyyk,228,Rebel Alliance,4
5,Obi-Wan Kenobi,Human,Stewjon,182,Jedi Order,7
6,Darth Vader,Human,Tatooine,202,Galactic Empire,1
7,Yoda,Yoda's species,Unknown,66,Jedi Order,8
8,R2-D2,Droid,Naboo,96,Independent,5
9,Padmé Amidala,Human,Naboo,165,Galactic Republic,5
10,Mace Windu,Human,Haruun Kal,188,Galactic Republic,None


## 📊 Understanding Normalization

**Normalization** is the process of organizing data to reduce redundancy.

### Before (One Table):

```
id | name          | homeworld | climate | population
1  | Luke Skywalker| Tatooine  | arid    | 200000
2  | Darth Vader   | Tatooine  | arid    | 200000
```

❌ **Problem:** "Tatooine", "arid", "200000" stored twice (duplication!)

### After (Two Tables):

```
characters:
id | name          | homeworld_id
1  | Luke Skywalker| 1
2  | Darth Vader   | 1

planets:
id | name     | climate | population
1  | Tatooine | arid    | 200000
```

✅ **Benefit:** Planet data stored once, referenced multiple times!

### Benefits of Normalization:

1. **No duplication** - Store each fact once
2. **Consistency** - One source of truth
3. **Easier updates** - Change data in one place
4. **Data integrity** - Foreign keys enforce valid relationships


## 🚗 Part 3: Create Vehicles Table

Let's create a vehicles table for starships and speeders.


In [15]:
%%sql
-- Insert planet data
CREATE TABLE IF NOT EXISTS vehicles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    model TEXT,
    vehicle_class TEXT,
    manufacturer TEXT
);

[]

### Insert Vehicles


In [16]:
%%sql
-- Insert vehicle data
INSERT INTO vehicles (name, model, vehicle_class, manufacturer) VALUES
    ('X-wing', 'T-65 X-wing', 'Starfighter', 'Incom Corporation'),
    ('Millennium Falcon', 'YT-1300 light freighter', 'Light freighter', 'Corellian Engineering Corporation'),
    ('TIE Fighter', 'Twin Ion Engine Fighter', 'Starfighter', 'Sienar Fleet Systems'),
    ('Imperial Speeder Bike', '74-Z speeder bike', 'Speeder', 'Aratech Repulsor Company'),
    ('Snowspeeder', 'T-47 airspeeder', 'Airspeeder', 'Incom Corporation'),
    ('Lambda Shuttle', 'Lambda-class shuttle', 'Transport', 'Sienar Fleet Systems'),
    ('AT-AT', 'All Terrain Armoured Transport', 'Assault walker', 'Kuat Drive Yards'),
    ('Jedi Starfighter', 'Delta-7 Aethersprite', 'Starfighter', 'Kuat Systems Engineering');


[]

### View Vehicles


In [17]:
%%sql
-- View the planets
SELECT * FROM vehicles

id,name,model,vehicle_class,manufacturer
1,X-wing,T-65 X-wing,Starfighter,Incom Corporation
2,Millennium Falcon,YT-1300 light freighter,Light freighter,Corellian Engineering Corporation
3,TIE Fighter,Twin Ion Engine Fighter,Starfighter,Sienar Fleet Systems
4,Imperial Speeder Bike,74-Z speeder bike,Speeder,Aratech Repulsor Company
5,Snowspeeder,T-47 airspeeder,Airspeeder,Incom Corporation
6,Lambda Shuttle,Lambda-class shuttle,Transport,Sienar Fleet Systems
7,AT-AT,All Terrain Armoured Transport,Assault walker,Kuat Drive Yards
8,Jedi Starfighter,Delta-7 Aethersprite,Starfighter,Kuat Systems Engineering


## 🔀 Part 4: Many-to-Many Relationships

### The Problem:

- Luke flies multiple vehicles (X-wing, Snowspeeder)
- Multiple characters fly the Millennium Falcon (Han, Chewie)
- One character → Many vehicles
- One vehicle → Many characters
- This is a **many-to-many** relationship!

### The Solution: Junction Table

A **junction table** (also called linking table or associative table) connects two tables in a many-to-many relationship.

**Structure:**
```
character_vehicles:
character_id | vehicle_id
1            | 1          (Luke → X-wing)
1            | 5          (Luke → Snowspeeder)
3            | 2          (Han → Millennium Falcon)
4            | 2          (Chewie → Millennium Falcon)
```


### Create Junction Table


In [19]:
%%sql
-- Insert character data with planet foreign keys
CREATE TABLE IF NOT EXISTS character_vehicles (
    character_id INTEGER NOT NULL,
    vehicle_id INTEGER NOT NULL,
    PRIMARY KEY (character_id, vehicle_id),
    FOREIGN KEY (character_id) REFERENCES characters(id),
    FOREIGN KEY (vehicle_id) REFERENCES vehicles(id)
);

[]

**Explanation:**

- `character_id` references the `characters` table
- `vehicle_id` references the `vehicles` table
- `PRIMARY KEY (character_id, vehicle_id)` ensures each pairing is unique (can't link Luke to X-wing twice)
- Both columns are foreign keys to their respective tables


### Link Characters to Vehicles

**Important:** Make sure you use the correct IDs from your tables!


In [29]:
%%sql
-- Create the character_vehicles junction table
INSERT INTO character_vehicles (character_id, vehicle_id) VALUES
    (1, 1),
    (1, 5),
    (3, 2),
    (4, 2),
    (5, 8),
    (6, 3),
    (6, 6),
    (8, 1),
    (8, 8)

(sqlite3.IntegrityError) UNIQUE constraint failed: character_vehicles.character_id, character_vehicles.vehicle_id
[SQL: -- Create the character_vehicles junction table
INSERT INTO character_vehicles (character_id, vehicle_id) VALUES
    (1, 1),
    (1, 5),
    (3, 2),
    (4, 2),
    (5, 8),
    (6, 3),
    (6, 6),
    (8, 1),
    (8, 8)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


### View Character-Vehicle Links


In [30]:
%%sql
-- View the junction table schema
SELECT * FROM character_vehicles

character_id,vehicle_id
1,1
1,5
3,2
4,2
5,8
6,3
6,6
8,1
8,8


## 🔍 Part 5: Verify Relationships

### Count Characters per Planet


In [33]:
%%sql
-- Insert relationships: Luke flies an X-wing
SELECT p.name AS planet, COUNT(c.id) AS population
FROM planets p
LEFT JOIN characters c ON p.id = c.homeworld_id
GROUP BY p.name
ORDER BY population DESC;

planet,population
Naboo,2
Tatooine,2
Alderaan,1
Corellia,1
Kashyyyk,1
Stewjon,1
Unknown,1
Hoth,0


**Note:** We're using a JOIN here (Lesson 6 topic), but it helps verify our relationships!

### Count Vehicles per Character


In [34]:
%%sql
-- View the relationships
SELECT c.name AS character, COUNT(cv.vehicle_id) AS vehicool
FROM characters c
LEFT JOIN character_vehicles cv ON c.id = cv.character_id
GROUP BY c.name
ORDER BY vehicool DESC;

character,vehicool
R2-D2,2
Luke Skywalker,2
Darth Vader,2
Obi-Wan Kenobi,1
Han Solo,1
Chewbacca,1
Yoda,0
Padmé Amidala,0
Mace Windu,0
Leia Organa,0


## 📋 Relationship Types Summary

### One-to-Many

**Setup:**
- Add foreign key column to the "many" table
- Points to primary key in the "one" table

**Example:** characters.homeworld_id → planets.id

### Many-to-Many

**Setup:**
- Create junction table
- Two foreign key columns (one for each table)
- Composite primary key on both columns

**Example:** character_vehicles (character_id, vehicle_id)


## 🎓 Practice Exercise

Add 2 more planets, 2 more vehicles, and create some new relationships!


### Exercise 1: Add More Planets


In [ ]:
%%sql
-- Exercise 1: Create a table for starships
INSERT INTO planets (name, climate, terrain, population) VALUES
    ('Endor', 'temperate', 'forests, mountains', 30000000),
    ('Bespin', 'temperate', 'gas giant', 6000000)

(sqlite3.IntegrityError) UNIQUE constraint failed: planets.name
[SQL: -- Exercise 1: Create a table for starships
INSERT INTO planets (name, climate, terrain, population) VALUES
    ('Endor', 'temperate', 'forests, mountains', 30000000),
    ('Bespin', 'temperate', 'gas giant', 6000000);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


### Exercise 2: Add More Vehicles


In [9]:
%%sql
-- Exercise 2: Insert starship data
INSERT INTO vehicles (name, model, vehicle_class, manufacturer) VALUES
    ('Slave I', 'Firespray-31', 'Patrol craft', 'Kuat Systems Engineering'),
    ('Naboo Starfighter', 'N-1 starfighter', 'Starfighter', 'Theed Palace Space Vessel Engineering Corps')

[]

### Exercise 3: Link New Vehicles to Characters


In [10]:
%%sql
-- Exercise 3: View your starships
INSERT INTO character_vehicles (character_id, vehicle_id) VALUES
    (2, 10)

[]

### Exercise 4: Verify Your Additions


In [11]:
%%sql
-- Challenge 1: Create species table
SELECT * FROM planets

id,name,climate,terrain,population
1,Tatooine,arid,desert,200000
2,Alderaan,temperate,"grasslands, mountains",2000000000
3,Hoth,frozen,"tundra, ice caves",None
4,Kashyyyk,tropical,"jungle, forests",45000000
5,Naboo,temperate,"grassy hills, swamps",4500000000
6,Corellia,temperate,"plains, urban",3000000000
7,Stewjon,temperate,grass,None
8,Unknown,None,None,None
9,Endor,temperate,"forests, mountains",30000000
10,Bespin,temperate,gas giant,6000000


In [12]:
%%sql
-- Challenge 1: Insert species data
SELECT * FROM vehicles

id,name,model,vehicle_class,manufacturer
1,X-wing,T-65 X-wing,Starfighter,Incom Corporation
2,Millennium Falcon,YT-1300 light freighter,Light freighter,Corellian Engineering Corporation
3,TIE Fighter,Twin Ion Engine Fighter,Starfighter,Sienar Fleet Systems
4,Imperial Speeder Bike,74-Z speeder bike,Speeder,Aratech Repulsor Company
5,Snowspeeder,T-47 airspeeder,Airspeeder,Incom Corporation
6,Lambda Shuttle,Lambda-class shuttle,Transport,Sienar Fleet Systems
7,AT-AT,All Terrain Armoured Transport,Assault walker,Kuat Drive Yards
8,Jedi Starfighter,Delta-7 Aethersprite,Starfighter,Kuat Systems Engineering
9,Slave I,Firespray-31,Patrol craft,Kuat Systems Engineering
10,Naboo Starfighter,N-1 starfighter,Starfighter,Theed Palace Space Vessel Engineering Corps


## 🐛 Common Errors & Troubleshooting

### Error: "FOREIGN KEY constraint failed"

**Problem:** Trying to insert a character_id or vehicle_id that doesn't exist.

**Solution:** Verify the IDs exist:


In [ ]:
%%sql
-- Challenge 2: Create films table
SELECT id, name FROM characters

id,name
1,Luke Skywalker
2,Leia Organa
3,Han Solo
4,Chewbacca
5,Obi-Wan Kenobi
6,Darth Vader
7,Yoda
8,R2-D2
9,Padmé Amidala
10,Mace Windu


In [ ]:
%%sql
-- Challenge 2: Insert film data
SELECT id, name FROM vehicles

id,name
1,X-wing
2,Millennium Falcon
3,TIE Fighter
4,Imperial Speeder Bike
5,Snowspeeder
6,Lambda Shuttle
7,AT-AT
8,Jedi Starfighter
9,Slave I
10,Naboo Starfighter


### Error: "UNIQUE constraint failed: planets.name"

**Problem:** Trying to insert a planet that already exists.

**Solution:** Check existing planets first:


In [ ]:
%%sql
-- Challenge 3: Create character_films junction table
SELECT name FROM planets

name
Alderaan
Bespin
Corellia
Endor
Hoth
Kashyyyk
Naboo
Stewjon
Tatooine
Unknown


### Wrong Foreign Key Values

**Problem:** `homeworld_id` doesn't match actual planet IDs.

**Solution:** Use subqueries to find correct IDs:


In [ ]:
%%sql
-- Challenge 3: Insert character-film relationships
SELECT id FROM planets WHERE name = 'Tatooine'

id
1


### Many-to-Many Confusion

**Remember:** 

**One-to-Many:**
- Use foreign key in the "many" table
- Example: characters.homeworld_id → planets.id

**Many-to-Many:**
- Use junction table with two foreign keys
- Example: character_vehicles (character_id, vehicle_id)

**Wrong Approach:**
```sql
-- ❌ DON'T DO THIS
ALTER TABLE characters ADD COLUMN vehicle_id INTEGER;
```

This only allows one vehicle per character!

**Correct Approach:**
```sql
-- ✅ DO THIS
CREATE TABLE character_vehicles (
    character_id INTEGER,
    vehicle_id INTEGER
);
```

This allows multiple vehicles per character and vice versa.


## 🎯 Challenge Problem (Optional)

**Task:** Design and create a `missions` table that tracks Star Wars missions. Each mission should have:
- A unique ID
- A name
- A location (foreign key to planets)
- A date
- A description

Then create a `character_missions` junction table to track which characters participated in which missions. Insert at least 3 missions and link characters to them.

**Requirements:**
- CREATE TABLE for missions
- CREATE TABLE for character_missions (junction table)
- INSERT 3+ missions
- INSERT 5+ character-mission links
- Verify with SELECT queries


In [ ]:
%%sql
-- Bonus Challenge 1: NOT NULL constraint demo


In [ ]:
%%sql
-- Bonus Challenge 2: UNIQUE constraint demo


In [ ]:
%%sql
-- Bonus Challenge 3: CHECK constraint demo


In [ ]:
%%sql
-- Bonus Challenge 4: DEFAULT value demo


In [ ]:
%%sql
-- Bonus Challenge 5: Foreign key constraint demo


## ✅ Checkpoint & Summary

### What You've Learnt

- ✅ Understand database normalization (avoid duplication)
- ✅ Create related tables with foreign keys
- ✅ Design one-to-many relationships (planet → characters)
- ✅ Build many-to-many relationships (characters ↔ vehicles)
- ✅ Use junction tables for many-to-many
- ✅ Insert data maintaining referential integrity
- ✅ Link records across tables using IDs
- ✅ Verify relationships with queries

### Key SQL Commands

| Command | Purpose | Example |
|---------|---------|---------|
| `UNIQUE` | Prevent duplicates | `name TEXT NOT NULL UNIQUE` |
| `FOREIGN KEY` | Link to another table | `FOREIGN KEY (homeworld_id) REFERENCES planets(id)` |
| `PRIMARY KEY` | Unique identifier | `id INTEGER PRIMARY KEY AUTOINCREMENT` |
| Junction Table | Many-to-many link | `character_vehicles(character_id, vehicle_id)` |
| Subquery | Find ID | `(SELECT id FROM planets WHERE name = 'Tatooine')` |

### Key Concepts

| Concept | Meaning |
|---------|---------|
| **Normalization** | Organizing data to reduce duplication |
| **Primary Key** | Unique identifier for rows in a table |
| **Foreign Key** | Field linking to another table's primary key |
| **One-to-Many** | One record relates to many records |
| **Many-to-Many** | Multiple records relate to multiple records |
| **Junction Table** | Links two tables in many-to-many relationship |
| **Referential Integrity** | Ensuring foreign keys reference valid records |

## 🎉 Excellent Work!

You've now built a proper relational database with multiple connected tables! In the next lesson, you'll learn how to retrieve data from multiple tables using JOINs.

**Ready to continue?** Open `lesson6_joins.ipynb`

---

## 💾 Git Commands (for reference)

```bash
git status
git add lessons/lesson5_schema.ipynb
git commit -m "Completed Lesson 5: Multiple tables and relationships"
git push
```
